In [4]:
import tabula
import textract
import re
import pymysql
from os import listdir
from os.path import isfile, join
import pandas as pd
import db_conn

In [5]:
positions = ['Goalkeeper', 'Defender', 'Midfield', 'Forward']

In [6]:
CONN = db_conn.get_connection

In [7]:
def insert_country(country):
    curs = CONN.cursor(pymysql.cursors.DictCursor)
    
    find_sql = '''select id from team where country like %s'''
    curs.execute(find_sql, ('%'+country+'%'))
    CONN.commit()
    result = curs.fetchone()
    
    if result is None:
        insert_sql = '''insert into team(country) values(%s)'''
        curs.execute(insert_sql, (country,))
        CONN.commit()
        curs.execute(find_sql, ('%'+country+'%'))
        result = curs.fetchone()

    return result

def extract_countries(game):
    country_one = ''
    country_two = ''
    if re.search('[a-zA-Z]+ [a-zA-Z]+ -', game) is not None:
        country_one = re.search('[a-zA-Z]+ [a-zA-Z]+ -', game).group().replace('-', '').strip()
    elif re.search('[a-zA-Z]+ - ', game) is not None:
        country_one = re.search('[a-zA-Z]+ - ', game).group().replace('-', '').strip()
    else:
        print('========Game Information Parsing Error=======')
        print(lines[6])
        raise ValueError
    
    if re.search('- [a-zA-Z]+ [a-zA-Z]+', game) is not None:
        country_two = re.search('- [a-zA-Z]+ [a-zA-Z]+', game).group().replace('-', '').strip()
    elif re.search('- [a-zA-Z]+', game) is not None:
        country_two = re.search('- [a-zA-Z]+', game).group().replace('-', '').strip()
    else:
        print('========Game Information Parsing Error=======')
        print(lines[6])
        raise ValueError
    
    return country_one, country_two

def insert_match(match_info):
    print(match_info)
    curs = CONN.cursor()
    
    find_sql = """select * from wc_match where id=%s"""
    curs.execute(find_sql % match_info['match_id'])
    result = curs.fetchone()
    print(result)
    if  result is not None:
        return result
    
    insert_sql = """INSERT INTO wc_match values(%s, %s, %s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (int(match_info['match_id']),
                              int(match_info['home_team']),
                              int(match_info['away_team']),
                              int(match_info['home_score']),
                              int(match_info['away_score']),
                              match_info['game_date'],
                              match_info['game_location']))
    CONN.commit()
    curs.execute(find_sql % match_info['match_id'])
    result = curs.fetchone()
    return result

In [8]:
def get_code_for_stat(type_value):
    curs = CONN.cursor()
    find_sql = """select code from stat_code where value like %s"""
    curs.execute(find_sql, type_value)#.replace('(', '\(').replace(')', '\)')
    result = curs.fetchall()
    return result

def insert_player_info(player_info):
    print(player_info)
    team_id = player_info['team_id']
    num = player_info['num']
    name = player_info['name']
    
    curs = CONN.cursor()
    find_sql = """select id from player where name like %s and num=%s"""
    curs.execute(find_sql, (name, num))
    player_id = curs.fetchone()
    if player_id is None:
        insert_sql = """insert into player(team_id, num, name) values(%s, %s, %s)"""
        curs.execute(insert_sql, (team_id, num, name))
        CONN.commit()
        curs.execute(find_sql, (name, num))
        player_id = curs.fetchone()
    return player_id

def insert_player_stat(player_stat):
#     print(player_stat)
    curs = CONN.cursor()
    insert_sql = """insert into player_stat (player_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (player_stat['player_id'], player_stat['match_id'], player_stat['stat_type'], player_stat['stat_value'], player_stat['half'],))
    CONN.commit()
    
def insert_team_stat(team_info):
#     print(team_info)
    curs = CONN.cursor()
    insert_sql = """insert into team_play_stat (country_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
    curs.execute(insert_sql, (team_info['team_id'], team_info['match_id'], team_info['stat_type'], team_info['stat_value'], team_info['half'],))
    CONN.commit()

In [9]:
def extract_row_value_insert_stat(value, stat_type, player_id, match_id, half):    
    code_search_key = line_match_code[stat_type]
    if type(code_search_key)==list:
        value = value.split('/')
        for i in range(len(code_search_key)):
            code_type = get_code_for_stat(code_search_key[i])[0]
            sub_value = value[i]
            if sub_value != '0':
                insert_player_stat({'player_id':player_id, 'match_id':match_id[0], 'stat_type':code_type, 'stat_value':sub_value, 'half':half})
    else:
        code_type = get_code_for_stat(code_search_key)[0]
        insert_player_stat({'player_id':player_id, 'match_id':match_id[0], 'stat_type':code_type, 'stat_value':value, 'half':half})

def extract_row_value_insert_team_stat(value, stat_type, team_id, match_id, half):    
    code_search_key = line_match_code[stat_type]
    if type(code_search_key)==list:
        value = value.split('/')
        for i in range(len(code_search_key)):
            code_type = get_code_for_stat(code_search_key[i])[0]
            sub_value = value[i]
            if sub_value != '0':
                insert_team_stat({'team_id':team_id, 'match_id':match_id[0], 'stat_type':code_type, 'stat_value':sub_value, 'half':half})
    else:
        code_type = get_code_for_stat(code_search_key)[0]
        insert_team_stat({'team_id':team_id, 'match_id':match_id[0], 'stat_type':code_type, 'stat_value':value, 'half':half})

In [10]:
# root_dir ='/Users/gracelee/football_worldcup' #group phase
root_dir ='/Users/gracelee/football_worldcup_knock'#knockout phase
onlyfiles = [f for f in listdir(root_dir) if isfile(join(root_dir, f))]
onlyfiles = [root_dir+'/'+x for x in onlyfiles if 'playerstatistics' in x]
# onlyfiles

In [27]:
def clear_variables():
    player_statistic = ''
    stats = ''
    lines = ''
    game = ''
    country_one = ''
    country_two = ''
    scores = ''
    game_date = ''
    game_location = ''
    match_id = ''
    country_one_id = ''
    country_two_id = ''
    team = ''
    team_id = ''
    number = ''
    name = ''
    minute = ''
    sec = ''
    play_in_game_df = pd.DataFrame()
    activity_in_game_df = pd.DataFrame()

In [28]:
line_match_code = {'Goal(s) scored':'%scored', 
                   'Shots':'Shots%', 
                   'Assist(s)':'Assist%',
                   'Offside(s)':'Offside%', 
                   'Save(s)':'Save%', 
                   'Yellow card':'Yellow%', 
                   '2Y+R/Red card':['%2Y%', '%Red card%'],
                   'Foul(s) committed':'Foul%', 
                   'Delivery/solo runs into the attacking third':['Delivery % attacking third', '%solo % attacking third%'],         
                   'Delivery/solo runs into the penalty area':['Delivery % penalty area', '%solo % penalty area%'],
                   'Tackles gaining/not gaining the ball':['Tackles gaining%', '%not gaining the ball%'],
                   'Tackles suffered losing/not losing the ball':['Tackles suffered losing%', '%not losing the ball'],
                   'Clearances completed/attempted':['Clearances completed%', '%attempted'],
                   'Total time played (mins)':'%Total time played%',
                   'Zone 1: 0-7 km/h time spent (%)':'%Zone 1%time spent%',
                   'Zone 2: 7-15 km/h time spent (%)':'%Zone 2%time spent%',
                   'Zone 3: 15-20 km/h time spent (%)':'%Zone 3%time spent%',
                   'Zone 4: 20-25 km/h time spent (%)':'%Zone 4%time spent%',
                   'Zone 5: >25 km/h time spent (%)':'%Zone 5%time spent%',
                   'Distance covered (metres)':'Distance covered%',
                   'Zone 1: 0-7 km/h distance covered (meters)':'%Zone 1%distance covered%',
                   'Zone 2: 7-15 km/h distance covered (meters)':'%Zone 2%distance covered%',
                   'Zone 3: 15-20 km/h distance covered (meters)':'%Zone 3%distance covered%',
                   'Zone 4: 20-25 km/h distance covered (meters)':'%Zone 4%distance covered%',
                   'Zone 5: >25 km/h distance covered (meters)':'%Zone 5%distance covered%',
                   'Top speed (km/h)':'%Top speed%',
                   'Sprints':'%Sprints%'
                  }

In [29]:
need_to_adjust = []

In [31]:
CONN = get_connection()

#Get player and game information
# for file in onlyfiles:
for z in range(30, len(onlyfiles)):
# for z in range(15, len(onlyfiles)):
    print('file z::: %s' % str(z))
    clear_variables()
    is_aet = False
    player_statistic = textract.process(onlyfiles[z])
    player_statistic = str(player_statistic)
    stats = player_statistic.split('2018 FIFA World Cup Russia')
    
    for j in range(1, len(stats)):
#     for j in range(1, 2):
        lines = stats[j].split('\\n')

        ##Game Info
        if lines[6].find('\(')>=0:
            game = lines[6][:lines[6].find('\(')]
        else:
            game = lines[6]
        if game.find('AET')>=0:
            game = game.replace('AET', '')
            is_aet = True
        country_one, country_two = extract_countries(game)
        scores = game.replace(country_one, '').replace(country_two, '').replace('-', '').split('(')[0].strip()
        scores = scores.split(' : ')
    #    print('%d ::: %s %s %s %s' %(j, country_one, country_two, scores[0], scores[1]))

        game_date = re.search('[0-9]+ [A-Z]+ 2018 [0-9]+:[0-9]+', lines[7]).group().strip()
        match = re.search('# [0-9]+', lines[7]).group().strip().replace('#', '').strip()
        game_location = lines[7].replace(game_date, '').replace(match, '').replace('#', '').strip()
#         if re.search('[a-zA-Z]+ \/ [a-zA-Z]+ [a-zA-Z]+ \/ [A-Z]+', lines[7]) is None:
#             game_location = re.search('[a-zA-Z]+ \/ [a-zA-Z]+ [a-zA-Z]+ [a-zA-Z]+ \/ [A-Z]+', lines[7]).group().strip().replace('/',',')
#         else:
#             game_location = re.search('[a-zA-Z]+ \/ [a-zA-Z]+ [a-zA-Z]+ \/ [A-Z]+', lines[7]).group().strip().replace('/',',')        

        if j==1:
            country_one_id = insert_country(country_one)
            country_two_id = insert_country(country_two)

            match_id = insert_match({'match_id':match,
                          'home_team':country_one_id['id'], 
                          'away_team':country_two_id['id'], 
                          'home_score':scores[0], 
                          'away_score':scores[1],
                          'game_date':game_date,
                          'game_location':game_location})
        #team
        team = lines[9]
        team_id = country_one_id['id'] if (team==country_one or lines[11]==country_one) else country_two_id['id']
        #player info
        if lines[10]!='':
            number = lines[10].split(' ')[0]
            name = ' '.join(lines[10].split(' ')[1:])
        elif lines[12]!='':
            number = lines[12].split(' ')[0]
            name = ' '.join(lines[12].split(' ')[1:])
            
        player_id = insert_player_info({'team_id':team_id,
                           'num':number,
                           'name':name})
        
#         if player_id[0]!=604 and player_id[0]!=433 and player_id[0]!=585 and player_id[0]!=512:
#             continue
        #play time
#         minute = re.search('[0-9]+',lines[12].split("\'")[0]).group()
#         sec = re.search('[0-9]+',lines[12].split("\'")[1]).group()
    #    print('%s : %s' %(minute, sec))
#         insert_player_stat({'match_id':match_id[0], 'player_id':player_id, 'stat_type':'TTP', 'stat_value':'%s\'%s\"' % (minute, sec), 'half':'total'})

        #substitution
        substitution = ''
        if len(lines[13].strip())>0:
            substitution = lines[13].strip()
#             print(substitution)
            insert_player_stat({'match_id':match_id[0], 'player_id':player_id, 'stat_type':'SUB', 'stat_value':substitution, 'half':'total'})

        position = ''
        position = lines[14].strip()
        if lines[14] not in positions:
            position = lines[15].strip()
            if lines[15] not in positions:
                position = lines[17].strip()
                if lines[17] not in positions:
                    position = lines[18].strip()
        print(str(j) + ' position ::' + position)
        insert_player_stat({'match_id':match_id[0], 'player_id':player_id, 'stat_type':'POS', 'stat_value':position, 'half':'total'})

        df = tabula.read_pdf(onlyfiles[z], multiple_tables=True, pages=j)
#         print('page ::: %s' %str(j))
        #, x1=26.999669828796403,x2=566.4721963790894,y1=420.0444706726074,y2=595.6520793151855)
        play_in_game_df = df[0]#df[(j-1)*2]
        activity_in_game_df = df[1]#df[j*2-1]
        
        if is_aet:
            if len(play_in_game_df.columns)==3:
                play_in_game_df.columns = ['type', 'team_total', 'team_avg']
                print('Extra time')
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                print(play_in_game_df)
            elif len(play_in_game_df.columns)==5:
                play_in_game_df.columns = ['type', 'extra_time','total', 'team_total', 'team_avg']
                print('Extra time')
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                print(play_in_game_df)
            elif len(play_in_game_df.columns)==6:
                play_in_game_df.columns = ['type', '1st_half', '2nd_half', 'total', 'team_total', 'team_avg']
                print('Extra time')
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                print(play_in_game_df)
            else:
                play_in_game_df.columns = ['type', '1st_half', '2nd_half', 'extra_time', 'total', 'team_total', 'team_avg']

            if len(activity_in_game_df.columns)==5:
                activity_in_game_df.columns = ['type', 'extra_time','total', 'team_total', 'team_avg']
                print('Extra time')
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                print(activity_in_game_df)
            elif len(activity_in_game_df.columns)==6:
                activity_in_game_df.columns = ['type', '1st_half', '2nd_half', 'total', 'team_total', 'team_avg']
                print('Extra time')
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                print(activity_in_game_df)
            else:
                activity_in_game_df.columns = ['type', '1st_half', '2nd_half', 'extra_time', 'total', 'team_total', 'team_avg']
        else:    
            if len(play_in_game_df.columns)==1:
                print('No play_in_game_df statistic information')
                #japan columnbia information is missing for japanese players for match 16
            elif len(play_in_game_df.columns)==3:
    #             print('Empty column in Table!!!!!')
    #             print(substitution)
    #             print({'match_id':match_id[0], 'player_id':player_id, 'name':name})

                play_in_game_df.columns = ['type', 'team_total', 'team_avg'] 

                need_to_adjust.append(onlyfiles[z])
    #             print(play_in_game_df)
            elif len(play_in_game_df.columns)==5:
                print('Empty column in Table!!!!!')
                print(substitution)
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})

                if substitution.lower().find('substituted by')>=0:
                    play_in_game_df.columns = ['type', '1st_half', 'total', 'team_total', 'team_avg'] 
                else:
                    play_in_game_df.columns = ['type', '2nd_half', 'total', 'team_total', 'team_avg']

                need_to_adjust.append(onlyfiles[z])
                print(play_in_game_df)
            else:    
                play_in_game_df.columns = ['type', '1st_half', '2nd_half', 'total', 'team_total', 'team_avg']

            if len(activity_in_game_df.columns)==1:
                print('No activity_in_game_df statistic information')
                continue
            elif len(activity_in_game_df.columns)==5:
                print('Empty column in Table!!!!!')
                print(substitution)
                print({'match_id':match_id[0], 'player_id':player_id, 'name':name})
                if substitution.lower().find('substituted by')>=0:
                    activity_in_game_df.columns = ['type', '1st_half', 'total', 'team_total', 'team_avg']
                else:
                    activity_in_game_df.columns = ['type', '2nd_half', 'total', 'team_total', 'team_avg']
                need_to_adjust.append(onlyfiles[z])
                print(activity_in_game_df)
            else:
                activity_in_game_df.columns = ['type', '1st_half', '2nd_half', 'total', 'team_total', 'team_avg']
    #         print(play_in_game_df)  

        for index, row in play_in_game_df.iterrows():
            stat_type = ''

            stat_type = row['type'].strip()

            if '1st_half' in list(play_in_game_df.columns) and pd.notna(row['1st_half']):
                extract_row_value_insert_stat(row['1st_half'], stat_type, player_id, match_id, 1)

            if '2nd_half' in list(play_in_game_df.columns) and pd.notna(row['2nd_half']):
                extract_row_value_insert_stat(row['2nd_half'], stat_type, player_id, match_id, 2)
            
            if 'extra_time' in list(play_in_game_df.columns) and pd.notna(row['extra_time']):
                extract_row_value_insert_stat(row['extra_time'], stat_type, player_id, match_id, 'extra')

            if j==1:
                if pd.notna(row['team_total']):
                    extract_row_value_insert_team_stat(row['team_total'], stat_type, team_id, match_id, 'total')
                if pd.notna(row['team_avg']):
                    extract_row_value_insert_team_stat(row['team_avg'], stat_type, team_id, match_id, 'avg')

#         print(activity_in_game_df)    
        for index, row in activity_in_game_df.iterrows():
            stat_type = ''

            stat_type = row['type'].strip()

            if '1st_half' in list(activity_in_game_df.columns) and pd.notna(row['1st_half']):
                extract_row_value_insert_stat(row['1st_half'], stat_type, player_id, match_id, 1)

            if '2nd_half' in list(activity_in_game_df.columns) and pd.notna(row['2nd_half']):
                extract_row_value_insert_stat(row['2nd_half'], stat_type, player_id, match_id, 2)

            if 'extra_time' in list(activity_in_game_df.columns) and pd.notna(row['extra_time']):
                extract_row_value_insert_stat(row['extra_time'], stat_type, player_id, match_id, 'extra')

            if pd.notna(row['total']):
                extract_row_value_insert_stat(row['total'], stat_type, player_id, match_id, 'total')

            if j==1:
                if pd.notna(row['team_total']):
                    extract_row_value_insert_team_stat(row['team_total'], stat_type, team_id, match_id, 'total')
                if pd.notna(row['team_avg']):
                    extract_row_value_insert_team_stat(row['team_avg'], stat_type, team_id, match_id, 'avg')
                
    a = input('Next File?')
    if a=='y':
        continue
need_to_adjust = list(set(need_to_adjust))

file z::: 30
{'match_id': '64', 'home_team': 9, 'away_team': 15, 'home_score': '4', 'away_score': '2', 'game_date': '15 JUL 2018 18:00', 'game_location': 'Moscow / Luzhniki Stadium / RUS'}
None
{'team_id': 15, 'num': '23', 'name': 'Danijel SUBASIC'}
1 position ::Goalkeeper
{'team_id': 15, 'num': '2', 'name': 'Sime VRSALJKO'}
2 position ::Defender
{'team_id': 15, 'num': '3', 'name': 'Ivan STRINIC'}
3 position ::Defender
{'team_id': 15, 'num': '4', 'name': 'Ivan PERISIC'}
4 position ::Forward
{'team_id': 15, 'num': '6', 'name': 'Dejan LOVREN'}
5 position ::Defender
{'team_id': 15, 'num': '7', 'name': 'Ivan RAKITIC'}
6 position ::Midfield
{'team_id': 15, 'num': '10', 'name': 'Luka MODRIC'}
7 position ::Midfield
{'team_id': 15, 'num': '11', 'name': 'Marcelo BROZOVIC'}
8 position ::Midfield
{'team_id': 15, 'num': '17', 'name': 'Mario MANDZUKIC'}
9 position ::Forward
{'team_id': 15, 'num': '18', 'name': 'Ante REBIC'}
10 position ::Forward
{'team_id': 15, 'num': '21', 'name': 'Domagoj VIDA'}


In [ ]:
for z in range(len(onlyfiles)):
#     print('file z::: %s' % str(z))
    is_aet = False
    player_statistic = textract.process(onlyfiles[z])
    player_statistic = str(player_statistic)
    stats = player_statistic.split('2018 FIFA World Cup Russia')
    
    for j in range(1, len(stats)):
        lines = stats[j].split('\\n')

        ##Game Info
        if lines[6].find('\(')>=0:
            game = lines[6][:lines[6].find('\(')]
        else:
            game = lines[6]
        if game.find('AET')>=0:
            game = game.replace('AET', '')
            is_aet = True
        match = re.search('# [0-9]+', lines[7]).group().strip().replace('#', '').strip()
       
        if j==1:
            print('!!!!!'+match)
            print(len(stats))